In [5]:
import pandas as pd
import numpy as np
import os
import time, datetime
import math
import random
from sklearn import preprocessing
import torch
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.classifier import StackingCVClassifier
from scipy.stats import norm, skew
from scipy import stats
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.impute import SimpleImputer
from scipy.misc import derivative
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import linear_model, datasets
from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import lightgbm
from sklearn import svm
from sklearn.neural_network import MLPRegressor,MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
import random
from sklearn.metrics import roc_auc_score
from scipy import optimize
from scipy import special
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn import preprocessing
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#2020-11-02 18:51:24.674845
#67884674845
def get_time_stamp(time):
    hh = int(time[11:11+2])
    mm = int(time[14:14+2])
    ii = int(time[17:17+2])
    res = hh*3600+mm*60+ii
    for i in range(20,len(time)):
        res=res*10+ord(time[i])-ord('0')
    return res

In [6]:
train_data_label = pd.read_csv('CCF2021_run_label_Train.csv')
label_p_id = train_data_label['人员编号']
label_c_id = train_data_label['特征码']
answer = {label_p_id[i]:label_c_id[i] for i in range(len(label_p_id))}

In [23]:
# train_data = pd.read_csv('train_feature.csv')
# test_data = pd.read_csv('test_feature.csv')
train_data = pd.read_csv('/content/drive/MyDrive/ccf data/train_feature_new.csv')
test_data = pd.read_csv('/content/drive/MyDrive/ccf data/test_feature_new.csv')

In [24]:
train_data.shape,test_data.shape

((398338, 20), (196382, 19))

In [ ]:
'''
cols = train_data.columns.tolist()
cols.remove('c_id')
cols.remove('p_id')
cols.remove('label')
for col in cols:
    Min = min(float(train_data[col].min()),float(test_data[col].min()))
    Max = max(float(train_data[col].max()),float(test_data[col].max()))
    train_data.loc[:,col] = (train_data.loc[:,col]-Min)/(Max-Min)
    test_data.loc[:,col] = (test_data.loc[:,col]-Min)/(Max-Min)
'''

"\ncols = train_data.columns.tolist()\ncols.remove('c_id')\ncols.remove('p_id')\ncols.remove('label')\nfor col in cols:\n    Min = min(float(train_data[col].min()),float(test_data[col].min()))\n    Max = max(float(train_data[col].max()),float(test_data[col].max()))\n    train_data.loc[:,col] = (train_data.loc[:,col]-Min)/(Max-Min)\n    test_data.loc[:,col] = (test_data.loc[:,col]-Min)/(Max-Min)\n"

In [ ]:
train_data

In [ ]:
test_data

In [25]:
p_ids = []
def init(data):
    global p_ids
    p_ids = data['p_id'].unique().tolist()

In [26]:
init(train_data)

In [27]:
random.seed(2021)
random.shuffle(p_ids)

In [28]:
cross_size = 5
n = len(p_ids)//cross_size + 1
p_data = [p_ids[i:i+n] for i in range(0, len(p_ids), n)]
def cross_val(model):
    global cross_val,p_ids
    acc = 0.0
    for cross in range(cross_size):
        train_p_ids = []
        for i in range(cross_size):
          if i != cross:
            train_p_ids = train_p_ids + p_data[i] 
        test_p_ids = p_data[cross]
        train_index = []
        test_index = []
        # for p_id in p_ids:
        #     if p_id not in train_p_ids:
        #         test_p_ids.add(p_id)
        train_index = train_data[train_data['p_id'].isin(train_p_ids)].index.tolist()
        test_index = train_data[train_data['p_id'].isin(test_p_ids)].index.tolist()
        small_train_data = train_data.iloc[train_index].drop(['c_id','p_id'],axis=1)
        small_test_data = train_data.iloc[test_index]
        test_c_id = small_test_data['c_id'].tolist()
        test_p_id = small_test_data['p_id'].tolist()
        small_test_data.drop(['c_id','p_id'],axis=1,inplace=True)
        model.fit(small_train_data.drop(['label'],axis=1),small_train_data['label'])
        predict_proba_y = model.predict_proba(small_test_data.drop(['label'],axis=1))[:,1:2]
        tmp_answer = {}
        for i in range(len(test_c_id)):
            proba = predict_proba_y[i][0]
            c_id = test_c_id[i]
            p_id = test_p_id[i]
            if proba > tmp_answer.get(p_id,(-1,-1))[1]:
                tmp_answer[p_id] = (c_id,proba)
        true = 0
        for p_id in tmp_answer:
            if tmp_answer[p_id][0] == answer[p_id]:
                true += 1
        print(true/len(test_p_ids))
        acc += true/len(test_p_ids)
    print('acc: ',acc/cross_size)
    # return -acc/cross_size

In [35]:
%%time
lgb=lightgbm.LGBMClassifier( objective='binary',
                        # metric='f1_score',
                        learning_rate=0.02,
                        n_estimators=500,
                        boosting= "gbdt",
                        max_depth=10,
                        num_leaves=400,
                        feature_fraction = 0.9,  
                        bagging_fraction = 0.95,  
                        bagging_freq = 5, 
                        reg_alpha = 0.02,
                        reg_lambda = 0.05,
                        min_data_in_leaf = 20,
                        )
params =  {'bagging_fraction': 0.9325837890849541, 'bagging_freq': 5, 'boosting_type': 'gbdt', 'feature_fraction': 0.7015355243328362, 'learning_rate': 0.06046174903181386, 'max_depth': 8, 'min_data_in_leaf': 15, 'min_gain_to_split': 0.6093427327673105, 'n_estimators': 500, 'num_leaves': 127, 'reg_alpha': 0.1949793976192661, 'reg_lambda': 0.960591321518608}
params = {'learning_rate': 0.017737413271505673, 'boosting_type': 'gbdt', 'objective': 'binary', 'num_leaves': 63, 'max_depth': 10, 'n_estimators': 500, 'min_data_in_leaf': 34, 'min_gain_to_split': 0.188661079484727, 'reg_alpha': 0.4248068068953985, 'reg_lambda': 1.039347932519718, 'feature_fraction': 0.9715679990466433, 'bagging_fraction': 0.6816784267307361, 'bagging_freq': 5, 'verbose': -1}

lgb = lightgbm.LGBMClassifier(**params)
cross_val(lgb)

0.92
0.9
0.9125
0.9125
0.9093198992443325
acc:  0.9108639798488666
CPU times: user 2min 26s, sys: 610 ms, total: 2min 27s
Wall time: 2min 26s


In [33]:
lr = LogisticRegression(# class_weight={0:1,1:100},
                        max_iter=5000,
                        random_state=2018)
cross_val(lr)

0.8275
0.795
0.825
0.8075
0.8060453400503779
acc:  0.8122090680100756


In [ ]:
dtc = DecisionTreeClassifier(max_depth=5,
                            class_weight={0:1,1:1})
abc = AdaBoostClassifier(base_estimator = dtc,
                        n_estimators=500,
                        learning_rate=0.15,
                        random_state=2019)
cross_val(abc)

In [30]:
# abc.fit(train_data.drop(['c_id','p_id','label'],axis=1),train_data['label'])
model = lgb 
model.fit(train_data.drop(['c_id','p_id','label'],axis=1),train_data['label'])

LGBMClassifier(bagging_fraction=0.95, bagging_freq=5, boosting='gbdt',
               boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               feature_fraction=0.9, importance_type='split',
               learning_rate=0.02, max_depth=10, min_child_samples=20,
               min_child_weight=0.001, min_data_in_leaf=20, min_split_gain=0.0,
               n_estimators=500, n_jobs=-1, num_leaves=400, objective='binary',
               random_state=None, reg_alpha=0.02, reg_lambda=0.05, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [ ]:
###false

test_c_id = train_data['c_id']
test_p_id = train_data['p_id']
predict_proba_y = model.predict_proba(train_data.drop(['c_id','p_id','label'],axis=1))[:,1:2]
tmp_answer = {}
train_acc = 0
false_c_id = []
false_p_id = []
for i in range(len(test_c_id)):
    proba = predict_proba_y[i][0]
    c_id = test_c_id[i]
    p_id = test_p_id[i]
    if proba > tmp_answer.get(p_id,(-1,-1))[1]:
        tmp_answer[p_id] = (c_id,proba)
test_p_id = list(set(test_p_id))
for p_id in test_p_id:
    if tmp_answer[p_id][0] == answer[p_id]:
        train_acc += 1
    else:
        false_p_id.append(p_id)
        false_c_id.append(tmp_answer[p_id][0])
print(train_acc,train_acc/len(test_p_id))
false = pd.DataFrame({'false_pids':false_p_id,
                      'false_cids':false_c_id})
false.to_csv('false.csv',index=False)

1987 0.9949924887330996


In [31]:

test_c_id = test_data['c_id']
test_p_id = test_data['p_id']
predict_proba_y = model.predict_proba(test_data.drop(['c_id','p_id'],axis=1))[:,1:2]
tmp_answer = {}
for i in range(len(test_c_id)):
    proba = predict_proba_y[i][0]
    c_id = test_c_id[i]
    p_id = test_p_id[i]
    if proba > tmp_answer.get(p_id,(-1,-1))[1]:
        tmp_answer[p_id] = (c_id,proba)
test_p_id = list(set(test_p_id))
test_c_id = []
for p_id in test_p_id:
    test_c_id.append(tmp_answer.get(p_id,'-1')[0])
submit = pd.DataFrame({'人员编号':test_p_id,
                       '特征码':test_c_id})
submit.to_csv('output.csv',index=False)

In [32]:
print(pd.DataFrame({
        'column': test_data.columns[2:], 
        'importance': model.booster_.feature_importance(),
    }).sort_values(by='importance',ascending = False))

                     column  importance
14                 Min_time        9840
1          average_det_time        9615
4   unique_average_det_time        9054
10                dis_speed        7294
2                cid_number        7022
12              c_speed_min        6782
9                   c_speed        6107
8                   p_speed        5988
6                  p_number        4340
15           c_speed_change        2592
7                pid_number        1806
0                 pp_number        1719
3          unique_pp_number        1186
5                unique_lcs         964
13            dis_speed_min         786
16           p_speed_change         478
11              p_speed_min           0


In [ ]:
test_data.columns

Index(['c_id', 'p_id', 'pp_number', 'average_det_time', 'cid_number',
       'unique_pp_number', 'unique_average_det_time', 'unique_lcs'],
      dtype='object')

In [ ]:
df_best = pd.read_csv('/content/output_11_4_23.csv')

In [ ]:
df_best.sort_values('人员编号',inplace=True)
submit.sort_values('人员编号',inplace=True)
num = 0
for i,j in zip(df_best['特征码'],submit['特征码']):
    if i!=j:
        num += 1
print(num)

0


In [ ]:
submit 

,人员编号,特征码
0,P0836,C9QtWNpl
1,P0888,CPzIxnsQ
2,P0934,C1W6xPCp
3,P0616,CQirDy2k
4,P0165,C4tlnf7j
...,...,...
991,P0345,CwKUR8PT
992,P0386,CTUFCl7y
993,P0658,Cka5fx4V
994,P0890,CkzOBbqo


In [ ]:
# submit = pd.read_csv('/content/output_add_speed.csv')
df_se = pd.read_csv('/content/output_11_15.csv')

In [ ]:
num = 0
ansa = {}
ansb = {}
ansc = {}
for pi,pj,i,j in zip(df_best['人员编号'],submit['人员编号'],df_best['特征码'],submit['特征码']):
  ansa[pi] = i 
  ansb[pj] = j
for p,c in zip(df_se['人员编号'],df_se['特征码']):
  ansc[p] = c

p_ans = []
c_ans = []
all_diff = 0
for p in df_best['人员编号']:
  res = [ansa[p],ansb[p],ansc[p]]
  p_ans.append(p)
  rs = ansa[p]
  if ansa[p] != ansb[p] and ansa[p]!=ansc[p] and ansb[p]!=ansc[p]:
    rs = ansa[p] 
    all_diff += 1
  else:
    rs = max(res,key = res.count)
  c_ans.append(rs)
  if rs != ansa[p]:
    num +=1
    print(p,ansa[p],rs)
print(num,all_diff)

P0927 C8nzP3gk CgAb8Ju4
P0806 COCNolEG C6jbQuYe
P0716 CxTrHqkl CSWFZ0LE
P0045 CEiaSN40 CL0ldUEC
P0418 CTxerhkd CWTVgGN1
P0306 Co4s6C0S CzaQW9xh
P0993 Cx3vlupn CGx3tXIi
P0876 CutQIJ16 CLS8EYq6
P0719 C5IS9sfA ChVKjnxW
P0287 ClMCfKV9 CxEf9bD3
P0182 CzKx4PUM CKrU67BY
P0020 CfEF4G1y CnMdqKxh
P0825 C5kc84PL Cz6edN1s
P0047 CizRADpn CvVN5qmw
P0494 C3ApMdE7 CGTSqfBz
P0874 CWji8wC9 CbNhD8xm
P0336 CoMxThPm C8dBCp3D
17 2


In [ ]:
sbmit = pd.DataFrame({'人员编号':p_ans,
                       '特征码':c_ans})
sbmit.to_csv('submit_2.csv',index=False)

In [21]:
lightgbm.__version__ 


'2.2.3'